<div dir="rtl" lang="he" xml:lang="he">


# אימון מודל ליניארי


add data from 


In [ ]:
#@markdown Import libraries
# %pip install torchviz
from torch import nn , optim
import torch
import matplotlib.pyplot as plt
from torchviz import make_dot

from typing import Callable,List,Tuple
from torch import Tensor
from torch.nn import Module
%matplotlib widget

In [ ]:
#@title plot functoins
#@markdown The functions are only intended to show the progress of the model
def plot_model(model,input_data,target,loss_function=None,list_loss=None):
  """
  evaluate input by model and print it
  model:one linear model
  input_data: (*,1) tnesor
  target: (*,1) tnesor
  loss_function: loss function
  list_loss: list of prives loss to plot it
  """
  y = model(input_data)      # to plot it we need change output to regular tensor
  W,b = None,None
  if type(model)==nn.Linear:
    W = model.weight.data.tolist()   #get W and b as float
    b = model.bias.data.item()

  fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10, 2.7))
  ax1.scatter(input_data,target, label='Y')
  ax1.plot(input_data,y.detach(), label=f'{W=},{b=:.3f}')
  ax1.legend()
  
  if list_loss is not None and loss_function:
    list_loss.append(loss_function(y,target).item())
    x_loss = range(len(list_loss))
    ax2.plot(x_loss,list_loss,label=f'{list_loss[-1]}')
    ax2.legend()
  fig.show()

def plot_3d_model(model,input_data,target,loss_function=None,list_loss=None):
  """
  evaluate input by model and print it
  model:one linear model
  input_data: (*,2) tnesor
  target: (*,1) tnesor
  loss_function: loss function
  list_loss: list of prives loss to plot it
  """
  x_grid,y_grid = torch.meshgrid(*input_data.T,indexing ="xy")
  pairs = torch.cartesian_prod(*input_data.T)
  out = model(pairs).detach()
  z_grid = out.view(x_grid.shape)




  fig = plt.figure(figsize=(15, 7))
  ax1,ax2 = fig.add_subplot(1, 2, 1, projection='3d'),fig.add_subplot(1, 2, 2)

  ax1.set_ylim(0,15)
  ax1.set_xlim(0,15)
  ax1.set_zlim(0,15)
 

  ax1.scatter(*input_data.T,target, label='Y',color="green")
  ax1.plot_surface(x_grid,y_grid,z_grid)
  ax1.legend()
  
  if list_loss is not None and loss_function:
    list_loss.append(loss_function(y,target).item())
    x_loss = range(len(list_loss))
    xlim = max(len(list_loss),10)
    ylim = max(max(list_loss),10)
    ax2.set_xlim(0,xlim)
    ax2.set_ylim(0,ylim)
    ax2.plot(x_loss,list_loss,label=f'{list_loss[-1]}')
    ax2.legend()
  fig.show()

def ploter(plot_func,model,x,target,loss_function,op_list_loss=None):
  loss = [] if op_list_loss is None else op_list_loss
  def f(op_loss=None):
    plot_func(model,x,target,loss_function,loss)
  return f


<div dir="rtl" lang="he" xml:lang="he">

# שיכבה לינארית של סקלר
שיכבה ליניארית של סקלר היא משוואה ישר פשוטה, נוכל לראות כיצד המודל שלנו מתקרב למשווה של הישר הזה. 



<div dir="rtl" lang="he" xml:lang="he">

## יצירת השיכבה

In [ ]:
linear = nn.Linear(1,1)
x = torch.rand(3,1)*10
print(f'''
{x=},
{linear(x)=}
''')

In [ ]:
W = linear.weight.data
b = linear.bias.data
print(f'''
{x=},
{W=},
{b=},
{x@W.T=},
{x@W.T+b=},
''')

<div dir="rtl" lang="he" xml:lang="he">

## אימון המודל
בשביל לבדוק את המודל, נקח את הפונקציה הפשוטה ביותר - 
$f(x)=x$
ונרעיש אותה קצת. נראה האם המודל שלנו יצליח להתקרב מספיק למשוואה הנדרשת 
 

In [ ]:
x = torch.arange(1,11,dtype=torch.float32 ).unsqueeze_(1)
target = torch.empty(10,1).uniform_(-1,1)+x
linear = nn.Linear(1,1)
plot_model(linear,x,target)

In [ ]:
loss_function = nn.MSELoss()
optimazer = optim.SGD(linear.parameters(),0.001)
f_plot = ploter(plot_model,linear,x,target,loss_function)

In [ ]:
for i in range(1000):
  y = linear(x)
  diff = loss_function(y,target)
  diff.backward()
  optimazer.step()
  if i %100==0:
    f_plot()

<div dir="rtl" lang="he" xml:lang="he">

# מודל שמקבל בשני מימדים
כעת נוכל לבנות מודל טפה יותר מורכב, שמקבל וקטור בגודל 2.
גם פה נקח את הקורדינטות על הקו ונרעיש אותם קצת. כמה מהר המודל יצליח להתקרב מספיק למשוואה?


In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.f1 = nn.Linear(2,1)
  def __call__(self,x):
    return self.f1(x)
  
model = Model()
loss_function = nn.MSELoss()
optimazer = optim.SGD(model.parameters(),0.000001)

<div dir="rtl" lang="he" xml:lang="he">

### יצירת דאטה
כמו בדוגמה הקודמת, ניצור דאטה פשוט. שהפונקציה היוצרת אותו היא מטריצת היחידה. 
נרעיש מעט את המידע שיש לנו, ונראה האם המודל מצליח לקרב את הפונקציה שלנו

In [ ]:
coordint =torch.arange(1,11,dtype=torch.float32 ).unsqueeze_(1)
target = torch.empty(10,1).uniform_(-1,1) + coordint
input_data = torch.cat([coordint,coordint],1)
plot_3d_model(model,input_data,target)

In [ ]:
loss_list = []
f_plot = ploter(plot_3d_model,model,input_data,target,loss_function,loss_list)

for i in range(100):
  y = model(input_data)
  diff = loss_function(y,target)
  loss_list.append(diff.item())
  diff.backward()
  optimazer.step()
  if i %10==0:
    f_plot()

<div dir="rtl" lang="he" xml:lang="he">

## השוואת פונקציות אופטימיזציה ופונקציות עלות
ניתן לראות במודל הקודם שתהליך הלמידה פה לא מתכנס, הוא נע בגלים ולא מגיע לתוצאה רצויה. נשווה בן כמה אופימזרים שיש, בשביל להבין מי מהם יותר טוב. 

<div dir="rtl" lang="he" xml:lang="he">


המבנה יהיה כזה:
ניצור 3 מודלים שונים, אחד פשוט של זיהוי תמונה, אחד של עיבוד טקסט ואת המודל הפשוט שלנו. לכל אחד מהם נתאים 3 אפשרויות לפונקציות אופטימיזציה, ונראה איך כל טכניקת אופטימיזציה מצליחה לגרום להתכנסות

בשביל שנוכל לבדוק בקלות את המודלים השונים, ניצור מחלקה חדשה `ModelTrainer`.
היא תכיל את המודל ואת פונקציית האימון שלו

In [ ]:
from torch import nn, Tensor
from torch.nn import Module
from typing import Callable, List, Tuple

LossFucntionType = Callable[[Tensor, Tensor], Tensor]
OptimazerType = Callable[[], None]
DataType = Tuple[Tensor, Tensor]
ParametersType = Tuple[List[float], float] 
GetParaetersType = Callable[[],parametersType]


In [ ]:
DATA = 0
LEABL = 1


class ModelTrainer:
    def __init__(
        self,
        model: Module,
        data: DataType,
        loss_function: LossFucntionType,
        optimizer: OptimazerType,
        get_parameters: GetParaetersType,
    ):
        self.model = model
        self.data = data
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.get_parameters = get_parameters
        self.len_data: int = len(data[DATA])
        self.counter: int = 0
        self.loss_list: List[float] = []

    def step(self, num_steps: int = 1)->None:
        for i in range(num_steps):
            input_data = self.data[DATA][self.counter]
            target = self.data[LEABL][self.counter]
            out = self.model(input_data)
            diff = self.loss_function(out, target)
            diff.backward()
            self.loss_list.append(diff.item())
            self.optimizer.step()
            self.counter = (self.counter + 1) % self.len_data  # repiat on data

    def get_less(self) -> List[float]:
        return self.loss_list


<div dir="rtl" lang="he" xml:lang="he">

## מודל ראשון: מודל ליניארי עם שיכבה אחת `LinModel`
נשתמש במודל הפשוט אותו בנינו,  יש לו שכבה אחת. 
זה מודל מאוד פשוט ונוכל בקלות להבין איך עובדים בו הדברים

In [ ]:
class LinModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.f1 = nn.Linear(2,1)
  def __call__(self,x):
    '''Activ the function'''
    return self.f1(x)
  def get_parameters(self, target: str) -> ParametersType:
    '''returns the importential parameters'''
    return self.f1.weight.tolist(), self.f1.bias.item()
lin_model_1 = LinModel()
lin_model_2 = LinModel()
lin_model_3 = LinModel()

In [ ]:

#loss functions and optimizers

sgd_lin_model_1 = optim.SGD(lin_model_1.parameters(),0.000001)
adam_lin_model_2 = optim.Adam(lin_model_2.parameters())
adam_lin_model_3 = optim.Adam(lin_model_3.parameters())


mse_loss = nn.MSELoss()
l1_loss = nn.L1Loss()